In [1]:
\l ../automl.q
.aml.loadfile`:init.q

In [2]:
n:10000
x:([]n?20;n?100f;asc n?100f)
yr:asc n?100f / regression
yc:20?0b       / binary classification
ym:n?`a`b`c   / multi classification
k:3           / number of folds
p:.2          / percentage of holdout
n:1           / number of repetitions
score_func:.ml.xv.fitpredict

In [3]:
tb:x
tgt:yc
mdls:.aml.models[`class;yc]
p:(::)
typ:`fresh

In [4]:
.aml.runexample[tb;tgt;typ;mdls;p]


Data preprocessing completed, starting feature creation



/home/deanna/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Feature creation completed, starting initial model selection - allow time for large datasets

Total features created = 434



/home/deanna/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.
2019-08-28 14:28:51.608811: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-08-28 14:28:51.612840: I tensorflow/core/common_runtime/process_util.cc:69] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.



Scores for all models, using .ml.accuracy

LogisticRegression        | 0.7
GaussianNB                | 0.7
SVC                       | 0.7
RandomForestClassifier    | 0.6333333
GradientBoostingClassifier| 0.5666667
KNeighborsClassifier      | 0.5666667
AdaBoostClassifier        | 0.5
LinearSVC                 | 0.4333333
MLPClassifier             | 0.3
BinaryKeras               | 0.3

Score for holdout predictions using best model - LogisticRegression

0.1666667

Feature impact calculated - see current directory for results

Model selection has been completed, continuing to the next step

